# 銘柄のスクリーニング
- 買う条件：以下をすべてみたすとき
  - 短期、中期、長期のすべての移動平均線が上向き
  - グランビルの法則が成立したとき（株価が中期または長期の移動平均線まで下がって反発したとき）
  - 上昇トレンドが3ヶ月以下
  - 移動平均線が上から短期、中期、長期の順
- 売る条件:
  - 株価が中期の移動平均線を下回ったとき
  - 5%以上下落したとき
  - (上昇の起点から)7日経過したとき

- 銘柄の選定
- 条件を満たすものをプログラムで選別
  - 流動性が高いものを選別
  -> その後目で見て判定 -> (データ作成もしてAI化?)

In [ ]:
from datetime import datetime
from pathlib import Path
import csv

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import stock

pio.renderers.default = "notebook"

In [ ]:
DAILY_DATA_DIR = Path.home() / "remote/gdrive/stock/data/daily/"
TARGET_LIST_CSV = stock.DATA_DIR / "margin_trade_target_list.csv"
daily_data_files = sorted(DAILY_DATA_DIR.glob("20220218_*.csv"))

In [ ]:
dfs = {}
for path in tqdm(daily_data_files):
    code = path.stem.split("_")[1]
    dfs[code] = pd.read_csv(path)

In [ ]:
with open(TARGET_LIST_CSV) as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    target_list = [row[0] for row in csv_reader]

### まずは過去1年間でショットガンのチャンスがどれくらいあるか検証する

In [ ]:
def calc_granville_points(close_prices, short=5, mid=20, long=60):
    short_ma = stock.chart.trend.moving_average(close_prices, window_size=short)
    mid_ma = stock.chart.trend.moving_average(close_prices, window_size=mid)
    long_ma = stock.chart.trend.moving_average(close_prices, window_size=long)

    max_idx = len(close_prices) - 1
    # 買いシグナルの探索
    buy_signals = []
    i = long + long
    while i < max_idx:
            # 移動平均線が下向きの場合は除外
        if short_ma[i] < 0 or mid_ma[i] < 0 or long_ma[i] < 0:
            i += 1
            continue

        # 移動平均線が短期 > 中期 順になっているか
        if short_ma[i] <= mid_ma[i]: # or mid_ma[i] <= long_ma[i]:
            i += 1
            continue

        buy_signals.append(i)
        while i <max_idx and short_ma[i] > mid_ma[i] and short_ma[i] >= 0 and mid_ma[i] >= 0 and long_ma[i] >= 0:
            i += 1
        # # 短期移動平均線が中期移動平均線で反発しているか
        # if short_ma[i - 2] >= short_ma[i - 1] and short_ma[i - 1] <= short_ma[i]:
        #     buy_signals.append(i)
        #     continue

        # # 短期移動線が中期移動線を上抜けしているか
        # if short_ma[i - 1] <= mid_ma[i - 1] and short_ma[i] >= mid_ma[i]:
        #     buy_signals.append(i)
        #     continue
        i += 1

    return buy_signals, short_ma, mid_ma, long_ma

In [ ]:
def calc_sell_point(buy_point_idx, close_prices, short_ma, mid_ma, long_ma, max_hold_days=9):
    """
    """
    # 上昇の起点の算出
    start_idx = buy_point_idx
    while close_prices[start_idx] > close_prices[start_idx - 1]:
        start_idx -= 1
    max_sell_idx = min(start_idx + max_hold_days, len(close_prices) - 1)

    if max_sell_idx <= buy_point_idx:
        return buy_point_idx

    # 買いポイントからの経過日数
    sell_point_idx = buy_point_idx + 1
    while sell_point_idx < max_sell_idx:
        # 売り条件を満たしている場合は売り
        # 移動平均線が下向き
        if short_ma[sell_point_idx] < 0 or mid_ma[sell_point_idx] < 0 or long_ma[sell_point_idx] < 0:
            break
        # 株価が短期移動線を下抜け
        if close_prices[sell_point_idx] < short_ma[sell_point_idx]:
            break
        sell_point_idx += 1
    return sell_point_idx

In [ ]:
def plot_shotgun(df, buy_pt_xs, buy_pt_ys, sell_pt_xs, sell_pt_ys, short, mid, long, short_ma, mid_ma, long_ma):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.0, row_heights=[0.7, 0.3])
    xdata = [datetime.strptime(dt, "%Y/%m/%d") for dt in df["day"].to_numpy()] # np.arange(len(df))
    fig.add_trace(go.Candlestick(x=xdata, open=df["open"], high=df["high"], low=df["low"], close=df["close"], name="candle"), row=1, col=1)
    fig.add_trace(go.Scatter(x=buy_pt_xs, y=buy_pt_ys, mode="markers", name="buy", marker=dict(size=10, color="blue")), row=1, col=1)
    fig.add_trace(go.Scatter(x=sell_pt_xs, y=sell_pt_ys, mode="markers", name="sell", marker=dict(size=10, color="yellow")), row=1, col=1)
    # fig.add_trace(go.Scatter(x=xdata, y=df["close"], name="close"), row=1, col=1)
    fig.add_trace(go.Scatter(x=xdata[short:], y=short_ma[short:], name="short_ma"), row=1, col=1)
    fig.add_trace(go.Scatter(x=xdata[mid:], y=mid_ma[mid:], name="mid_ma"), row=1, col=1)
    fig.add_trace(go.Scatter(x=xdata[long:], y=long_ma[long:], name="long_ma"), row=1, col=1)

    fig.add_trace(go.Bar(x=xdata, y=df["volume"], name="volume"), row=2, col=1)

    fig.update_layout(xaxis_rangeslider_visible=False, 
                    xaxis2_rangeslider_visible=True,
                    margin=go.layout.Margin(l=5, r=5, t=5, b=5, autoexpand=True),
                    )
    #fig.update_traces(mode="markers+lines", hovertemplate=None)
    fig.update_layout(hovermode="x unified")                  
    fig.update_traces(xaxis='x2')
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])  # 土日を除外
    fig.show()

In [ ]:
def shotgun(df, short=5, mid=20, long=60, with_plot=False):
    open_column_key="open"
    close_column_key="close"
    df = df.dropna(how="all", axis=1)
    if df.isnull().any().any():
        return np.array([])
    # 買いシグナルの探索
    buy_signals, short_ma, mid_ma, long_ma = calc_granville_points(df[close_column_key], short=short, mid=mid, long=long)
    buy_pt_ys = [df[open_column_key][idx + 1] for idx in buy_signals]  # 買いシグナルの次の日の始値で買う
    # 売りシグナルの探索
    sell_points = [calc_sell_point(buy_point_idx, df[close_column_key].to_numpy(), short_ma, mid_ma, long_ma) for buy_point_idx in buy_signals]
    sell_pt_ys = [df[close_column_key][idx] for idx in sell_points]

    if with_plot:
        buy_pt_xs = [datetime.strptime(df["day"][idx + 1], "%Y/%m/%d") for idx in buy_signals]
        sell_pt_xs = [datetime.strptime(df["day"][idx], "%Y/%m/%d") for idx in sell_points]
        plot_shotgun(df, buy_pt_xs, buy_pt_ys, sell_pt_xs, sell_pt_ys, short, mid, long, short_ma, mid_ma, long_ma)

    profit = np.array(sell_pt_ys) - np.array(buy_pt_ys)
    return profit

In [ ]:
short = 5
mid = 20
long = 100
targets = target_list[:10]  # 上位100銘柄のみ

profit_num = 0
profit_total = 0
profits = []
change = []
for i, code in enumerate(targets):
    #profit = shotgun(dfs, code, short=short, mid=mid, long=long, with_plot=i < 4)
    profit = shotgun(dfs[code], short=short, mid=mid, long=long, with_plot=False)
    if len(profit) > 0:
        profit_norm = profit.sum()  / dfs[code]["close"].mean()
        profits.append(profit_norm)
        change.append((dfs[code]["close"][len(dfs[code]) - 1] - dfs[code]["open"][0]) / dfs[code]["close"].mean())
        # print(code, profit_norm)
        if not np.isnan(profit_norm):
            profit_num += len(profit)
            profit_total += profit_norm

codes = targets
profits = np.array(profits)
change = np.array(change)
print("profit mean : {}, min : {} ({}), max : {} ({})".format(
    profit_total / profit_num, profits.min(), codes[profits.argmin()], profits.max(), codes[profits.argmax()]))
print("change mean : {}, min : {} ({}), max : {} ({})".format(
    change.mean(), change.min(), codes[change.argmin()], change.max(), codes[change.argmax()]))

In [ ]:
code = codes[0]
df = dfs[code]
profit = shotgun(df, short=short, mid=mid, long=long, with_plot=True)
profit = np.array(profit)

print("code = {}, profit = {}".format(code, profit.sum()))

plt.plot(np.cumsum(profit) / df["close"].mean())
plt.grid()

In [ ]:
code = codes[3]
csv_path = stock.DATA_DIR / "etfs"/ f"{code}.csv"
if not csv_path.exists():
    csv_path = stock.DATA_DIR / "nikkei225"/ f"{code}.csv"
df = pd.read_csv(csv_path)
df["day"] = df["timestamp"].map(lambda x: datetime.fromtimestamp(x).strftime("%Y/%m/%d"))
df["open"] = df["start"]
df["close"] = df["end"]

profit = shotgun(df, short=short, mid=mid, long=long, with_plot=True)
profit = np.array(profit)

print("code = {}, profit = {}, change {}".format(code, profit.sum(), df["close"][len(df) - 1] - df["open"][0]))

plt.plot(np.cumsum(profit) / df["close"].mean())
plt.grid()